In [9]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd  

In [10]:
engine = create_engine(connstr) 
def get_df(query):
    conn = engine.connect() 
    df = pd.read_sql(query,con=conn) 
    conn.close()
    return df

In [11]:
def id_split(s):
    s = s + '000'
    s = re.sub('[^\d]', '', s)  
    return s[:4] 
#print( id_split('5.1-4') )

In [12]:
def id_num(s):
    if s == '':
        return 0
    s = s.strip().lower().replace(' ', '')
    l = ['a','b','c','d','e','f','g','h']
    d = [.100, 1, 100, 10000]
    v = [0,0,0,0]
    for i, c in enumerate(l):
        s = s.replace(c,f'{i}') 
    l = s.split('.')
    for i, c in enumerate(l):
        v[i] =  int(c) / d[i] 
    return sum(v)  
print(id_num('1.1.1')) 
print(id_num('1.1.10.1')) 

11.01
11.1001


In [13]:
def normalizedf(df):
    df.drop( columns=[c for c in df.columns if '__' in c ] , inplace=True )   
    cols = list(df)   
    df[cols] = df[cols].astype(str)
    df['IDNUM'] = df['IDTEXT'].apply(id_num)
    df.sort_values('IDNUM',inplace=True)   
    df['QPK'] = range(28200, len(df)+28200 )  
    df['sortpos'] = range(1, len(df)+1)  
    return df

In [18]:
query = f""" \
    SELECT TOP 5000 
      IDTEXT AS IDTEXT 
    , PK_Question
    , PK_Question as [2021PK_Question]
    , QGROUP
    , '@FormName' AS  FormName
    , '@PK_QGroup' AS  FK_QuestionGroup
    , sortpos AS  sortpos 
    , NULL AS  FK_Question_Picklist
    , CONVERT(NVARCHAR(5), FK_PickListType) AS  FK_PickList 
    , ISNULL(FK_QuestionType, '') AS  FK_QuestionType
    , NULL AS  FK_InputType
    , CONVERT(NVARCHAR(5), FK_PickList) AS FK_PickListType
    , NULL AS  WarningQuestion 
    , NULL AS  ExternalLinkType
    , NULL AS  PK_ExternalLink
    , ISNULL(QText, '') AS  QuestionText
    , ISNULL(help_text, '') AS  help_text 
    FROM vwQuestions WHERE PK_FORM LIKE '2021-A-HVA'  
"""  
dfq  = get_df(query) 
dfq = normalizedf(dfq)
dfq.sort_values(['QGROUP','IDNUM'], inplace=True)
dfq.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfq.csv', index=False)  
dfq[:2] 

,IDTEXT,PK_Question,2021PK_Question,QGROUP,FormName,FK_QuestionGroup,sortpos,FK_Question_Picklist,FK_PickList,FK_QuestionType,FK_InputType,FK_PickListType,WarningQuestion,ExternalLinkType,PK_ExternalLink,QuestionText,help_text,IDNUM,QPK
0,0.1,22575,22575,2431,@FormName,@PK_QGroup,14,None,None,18,None,None,None,None,None,HVA Information,,1.0,28213
2,1,22577,22577,2431,@FormName,@PK_QGroup,15,None,None,9,None,None,None,None,None,What is the purpose of the HVA?,,10.0,28214


In [19]:
dfh = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='1') 
dfh = normalizedf(dfh) 
dfh['QuestionText'] = ''
dfh[0:2] 

,IDTEXT,FK_QuestionType,FK_PickListType,IDNUM,QPK,sortpos,QuestionText
0,1,9,nan,10.0,28200,1,
1,1.1,17,nan,11.0,28201,2,


In [20]:
df1 = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='ALL') 
df1 = normalizedf(df1) 
df1[1:2]  # .loc[ df1['IDTEXT'].str.contains('5.') ]

,IDTEXT,QuestionText,QGROUP,IDNUM,QPK,sortpos
1,1,What is the purpose of the HVA?,2431.0,10.0,28201,2


ppl = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='PurposePickList' )
l=[]
c=0
for i,r in ppl.iterrows():
    if i % 2==1: 
        continue
    c=c+1
    IDTEXT = f"1.1.{ c }"
    QuestionText = f"{ r['DisplayValue'] }"
    l.append( {'IDTEXT':f'{IDTEXT}', 'QuestionText': f"If {QuestionText}, describe:"})
ppl = pd.DataFrame(l)  
ppl.to_csv(r'C:\Users\Tim\Documents\1802\HVA\ppl.csv', index=False)  
ppl[0:2]


In [17]:
dfp = pd.read_csv(r'C:\Users\Tim\Documents\1802\HVA\HVAPurpose.csv' )
dfp['IDTEXT'] = dfp['IDTEXT'].apply(lambda s : s.replace('.a','.1'))
dfp['IDTEXT'] = dfp['IDTEXT'].apply(lambda s : s.replace('.b','.2'))
dfp = normalizedf(dfp)  
dfp[:]

,QuestionText,IDTEXT,IDNUM,QPK,sortpos
0,.GOV Domain Registration Systems,1.1.1,11.01,28200,1
1,Access to Data systems,1.1.2,11.02,28201,2
2,Agency support tool,1.1.3,11.03,28202,3
3,Business Intelligence Services,1.1.4,11.04,28203,4
4,Collaboration Tools (SharePoint),1.1.5,11.05,28204,5
5,"Collect, analyze, maintain and share data",1.1.6,11.06,28205,6
6,Consulate Database (VISA),1.1.7,11.07,28206,7
7,Cybersecurity Monitoring,1.1.8,11.08,28207,8
8,Cybersecurity tools/solutions,1.1.9,11.09,28208,9
9,Data Repository,1.1.10,11.10,28209,10


In [23]:
dfm = pd.merge(df1, dfp, how='outer', on=['IDTEXT', 'QuestionText'], suffixes=(None, '__')) 
dfm = normalizedf(dfm)
dfm.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfm.csv', index=False)  
dfm[0:2]  

,IDTEXT,QuestionText,QGROUP,IDNUM,QPK,sortpos
0,0.1,HVA Information,nan,1.0,28200,1
1,1,What is the purpose of the HVA?,2431.0,10.0,28201,2


In [58]:
dfa = dfm
dfa = normalizedf(dfa) 
for i,r in dfa.iterrows():
    v=dfa.loc[i, 'QuestionText']
    if v=='':
        dfa.loc[i, 'QuestionText']=dfa.loc[i, 'QuestionText_']
dfa.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfa.csv', index=False)   
dfa['QuestionText'].fillna('')
dfa[0:2] 

,IDTEXT,QuestionText,QGROUP,IDNUM,QPK,sortpos
0,0.1,HVA Information,nan,1.0,28200,1
1,1,What is the purpose of the HVA?,2431.0,10.0,28201,2


In [59]:
dff = pd.merge(dfa, dfq, how='left', on='IDTEXT', suffixes=('', '_y')) 
dff['FK_QuestionType'].fillna('9', inplace=True)
dff['FK_PickList'].fillna('NULL', inplace=True)
dff['FK_PickListType'].fillna('NULL', inplace=True)
dff['WarningQuestion'].fillna('NULL', inplace=True)
dff['PK_ExternalLink'].fillna('NULL', inplace=True)
dff['PK_Question']=dff['QPK']
dff['FK_InputType'].fillna('NULL', inplace=True) 
dff['help_text'].fillna('NULL', inplace=True) 
dff = dff.fillna('NULL')
dff  = dff.replace('None','NULL')
dff.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfa.csv', index=False)   
dff[:]

,IDTEXT,QuestionText,QGROUP,IDNUM,QPK,sortpos,PK_Question,2021PK_Question,QGROUP_y,FormName,...,FK_QuestionType,FK_InputType,FK_PickListType,WarningQuestion,ExternalLinkType,PK_ExternalLink,QuestionText_y,help_text,IDNUM_y,QPK_y
0,0.1,HVA Information,nan,1.00,28200,1,28200,22575,2431,@FormName,...,18,NULL,NULL,NULL,NULL,NULL,HVA Information,,1.0,28213.0
1,1,What is the purpose of the HVA?,2431.0,10.00,28201,2,28201,22577,2431,@FormName,...,9,NULL,NULL,NULL,NULL,NULL,What is the purpose of the HVA?,,10.0,28214.0
2,1.1,Please Select the Applicable Purpose Type:,2431.0,11.00,28202,3,28202,NULL,NULL,NULL,...,9,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
3,1.1.1,.GOV Domain Registration Systems,nan,11.01,28203,4,28203,NULL,NULL,NULL,...,9,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
4,1.1.2,Access to Data systems,nan,11.02,28204,5,28204,NULL,NULL,NULL,...,9,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,21.3,Please indicate the organization type for the ...,nan,213.00,28358,159,28358,22747,2435,@FormName,...,18,NULL,NULL,NULL,NULL,NULL,If 'Yes' to Internal to Agency then Is this sy...,,213.0,28299.0
159,21.4,Does the D/A for this HVA have an ISA/MOU with...,nan,214.00,28359,160,28359,22748,2435,@FormName,...,18,NULL,NULL,NULL,NULL,NULL,Is this interconnection/data exchange mission ...,,214.0,28300.0
160,21.5,Is this interconnection/data exchange mission ...,nan,215.00,28360,161,28360,22749,2435,@FormName,...,18,NULL,NULL,NULL,NULL,NULL,"Is this Interconnection Transmit, Receive, or ...",,215.0,28301.0
161,21.6,Is this Interconnection or data exchange a hig...,nan,216.00,28361,162,28361,NULL,NULL,NULL,...,9,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL,NULL


In [ ]:
#dff = pd.read_csv(r'C:\Users\Tim\Documents\1802\HVA\HVAPurpose.csv' )
#dff

In [40]:
def clean_text(row):
    # return the list of decoded cell in the Series instead 
    return [r.decode('unicode_escape').encode('ascii', 'ignore') for r in row]


In [60]:
query = f"""  SELECT TOP 1 
    PK_Question, FormName, FK_QuestionGroup,sortpos,identifier_text, FK_QuestionType
    , FK_InputType, FK_PickListType, WarningQuestion, ExternalLinkType, QuestionText, help_text 
    FROM  fsma_Questions 
    """  
df_qmeta = get_df(query) 
cols = ','.join(df_qmeta.columns)
INS = f'INSERT INTO fsma_Questions ({cols}) VALUES '
print(INS)

INSERT INTO fsma_Questions (PK_Question,FormName,FK_QuestionGroup,sortpos,identifier_text,FK_QuestionType,FK_InputType,FK_PickListType,WarningQuestion,ExternalLinkType,QuestionText,help_text) VALUES 


In [62]:
sql = "" 
for i, r in dff[:].iterrows(): 
    s = "(@QID, @FormName, @PK_QGroup, @SORD, N'@IDTEXT', @QTYPE, 1, @PKTYPE, 0, NULL, N'@QTEXT','@HTEXT'),"
    s = s.replace('@QID',  str(r['QPK']))
    s = s.replace('@SORD', str(i+1)) 
    s = s.replace('@QTYPE', r['FK_QuestionType']) 
    s = s.replace('@PKTYPE', r['FK_PickListType']) 
    s = s.replace('@IDTEXT', r['IDTEXT']) 
    s = s.replace('@QTEXT', r['QuestionText'].replace('\'', '`').replace('\ufffd', ''))  
    s = s.replace('@HTEXT', r['help_text'])  
    sql = sql + s + '\r'
sql = sql.replace("'NULL')", "NULL)")    
sql = sql.replace("N'NULL'", "N''")   
sql = sql.replace("None,", "NULL,")  
sql = f'{INS}\n {sql}\n' 
with open(fr'C:\Users\Tim\Documents\1802\HVA\update.sql', 'w') as f:
    f.write(sql)  



df = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='PurposePickList').fillna('NULL') 
cols = list(df) # Creates list of all column headers
df[cols] = df[cols].astype(str)
df['DisplayCode'] =  df['DisplayValue'].apply(lambda s: re.sub('[^A-Z0-9]','',s.upper()))
df['DisplayCode'] =  df['DisplayCode'].apply(lambda s: re.sub('[AEIOURST]','',s)[:8])
s =''
for i,r, in df.iterrows():
    PK_PickList=r['PK_PickList']
    DisplayCode=r['DisplayCode']
    DisplayValue=r['DisplayValue'] 
    if DisplayValue=='Other':
        DisplayCode='OTH'
    s = s + f"({PK_PickList},@PKT, N'{DisplayCode}',N'{DisplayValue}',{i}, GETDATE(), 1),\n"

print(s)
